## Loading Packages & Initialization 

In [1]:
rm(list=ls())

library(data.table)
library(tidyverse)
library(rJava)
library(RNetLogo)
library(lhs)

options(warn = -1)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.tab

In [2]:
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"
source(paste0(folder.path,"ThesisSetupCode.r"))

Is_Headless <- 1
nl.model <- "info_cascade_update_TDP_JPF_2020"

nl.path <- "/Users/ecemnaz.yildiz/Documents/NetLogo 6.0.4/Java"
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"

model.path <- paste0(folder.path, nl.model, ".nlogo")

if (Is_Headless == 0) {
    NLStart(nl.path, gui = TRUE, nl.jarname = "netlogo-6.0.4.jar")
    NLLoadModel(model.path)
} else {
    NLStart(nl.path, gui = FALSE, nl.jarname = "netlogo-6.0.4.jar", nl.obj = nl.model)
    NLLoadModel(model.path, nl.obj = nl.model)
}

options(warn = -1)


Attaching package: ‘plotly’

The following object is masked from ‘package:igraph’:

    groups

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa
Loading required package: gridExtra

Attaching package: ‘gridExtra’

The following object is masked from ‘package:randomForest’:

    combine

The following object is masked from ‘package:dplyr’:

    combine



In [3]:
model.type = "info_cascade_update"
# the path of data folder
data.path = paste0(folder.path,"data/")
# the path for outputs to be record
output.folder = paste0("outputs_AdS_cvar_TimeTrack_",model.type,"_",Sys.Date())
dir.create(file.path(folder.path, output.folder), showWarnings = FALSE)

outputs.path = paste0(folder.path,output.folder,"/")

# Read Me File to keep info about the output folder
ReadMe = paste0(outputs.path,"ReadMe_",model.type,".txt")

## Model Parameters & Functions

### Set model parameters

In [4]:
#### Model Parameters ####
## Set model parameters Number of replications for each instance
nofrep = 30

feature_names = c(    
    "max_links",
    "evidence",
    "sc-bel-prop",
    "prop-likelihood",
    "n_init_believers",
    "prior-mean",
    "prior-sd",
    "expertise_influence") 
    
feature_ranges = data.table(  feature   = feature_names
                            , min_range = c(2, 0, 0, 0, 0, 0, 0, 0)
                            , max_range = c(500, 100, 5, 1, 100, 1, 1, 1)
                           )

output_name = c("cl-prop-same")

# Number of input parameters of the agent-based model
nofparams = length(feature_names)

# set RF parameters
ntree = 300
mtry = 6
nperm = 5

### Set user parameters

In [8]:
#### User parameters ####
error_type = "RMSE"  # MAPE, BIAS

# choose the uncertainty measure
selection_metric <- "coefvar"  #, 'range' 
sample.type = selection_metric

# Number of iterations
iteration_budget = 11
metarep = c(1:1)

# Number of instances
unlabeled_ins = 30
test_ins = c(30) #c(100,200,300,400)
train_ins_oneshot = 30
train_ins_Ad = 30

# Set selection parameters
selected_ins = 5  #nofinstancesWillbeSelected in each step

# Set elimination parameters
h <- 1  # number of variables eliminated in each step

seed.focus = c(8)#,1,2,3,4,5,6,7,8,9,20)

## to be used in log entries
unlabeled.type = "refresh"

log_entry.sampling()

In [9]:
seed.focus

[1] 8

## Test Set

In [10]:
#### Test Sets ####
test_set = data.table()

for( t in test_ins){
    test_set.name= paste0(data.path,"test_set","_",model.type,"_",t,".csv")
    test_set_Sub <- fread(test_set.name)  
    
    test_set = rbind(test_set, data.table(size = t, test_set_Sub))
}

In [11]:
test_set

size,max_links,evidence,sc-bel-prop,prop-likelihood,n_init_believers,prior-mean,prior-sd,expertise_influence,output
30,153.52986,17.822423,3.5235470,0.99942973,21.415365,0.31318558,0.73333450,0.388069845,48.10765
30,238.41998,25.829850,1.6892055,0.21765580,66.338854,0.69404958,0.26549282,0.529586065,44.14544
30,496.77583,80.553581,0.5116790,0.43575215,92.243732,0.20535621,0.49550760,0.549731941,42.16805
30,261.28562,16.483629,3.3100315,0.05897293,15.308214,0.89228352,0.50357083,0.239288878,51.75538
30,421.92905,97.740826,3.6954132,0.63071798,43.870431,0.58688183,0.67234487,0.782341848,46.06913
30,362.21065,3.572477,4.4936570,0.88007035,99.618287,0.88094505,0.43866759,0.068812180,48.33022
30,308.35369,82.550522,1.4803079,0.11926421,44.761687,0.98306526,0.41839219,0.190624533,55.43060
30,370.10732,4.114588,2.6234796,0.34156773,81.519827,0.32036658,0.45222689,0.043802018,44.59081
30,209.78551,5.307582,0.7697447,0.85939482,18.812904,0.81556064,0.66006938,0.123153712,49.58548
30,187.56724,51.349104,3.3438063,0.61702112,20.374490,0.59864803,0.20708671,0.377884488,48.38556


## One-Shot Training Set

In [13]:
one_shot_data = upload_training_set(model.type,seed.focus,train_ins_oneshot)

In [14]:
one_shot_data

max_links,evidence,sc-bel-prop,prop-likelihood,n_init_believers,prior-mean,prior-sd,expertise_influence,output,seed
379.030421,33.093892,2.3776148,0.45018803,81.489234,0.25385494,0.55607425,0.44670360,42.97597,8
354.720810,53.770374,1.7306876,0.39097777,51.108860,0.30752116,0.74643066,0.49186844,45.43132,8
150.663956,52.282630,2.5397536,0.35723122,49.568888,0.46417893,0.20238519,0.59242957,45.53525,8
310.303729,37.939215,1.6481274,0.79260720,39.472428,0.41508232,0.49821941,0.54886696,46.41394,8
246.681021,69.824069,2.1237128,0.49174655,79.052799,0.66834203,0.87004269,0.41265428,43.12122,8
284.665309,2.769528,4.5423103,0.62618815,36.565307,0.37020836,0.42145825,0.70496254,46.85059,8
68.156233,17.351079,3.1303719,0.69802720,46.491671,0.35416725,0.72813793,0.35974346,45.94112,8
395.927951,72.373656,4.0555379,0.25140501,54.704373,0.74388255,0.07464179,0.76934667,45.96783,8
265.703127,79.724977,3.9051546,0.71919613,60.588566,0.21410364,0.32198382,0.32189261,47.28248,8
223.475741,60.791212,3.5232455,0.99764996,11.128286,0.93562664,0.33789561,0.36937890,53.15028,8


### One-shot Train & Test Metamodel

In [15]:
sample.type = "oneshot"

sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

iter = 1
for (i in seed.focus) {
    
    training_set = copy(one_shot_data[seed == i, .SD, .SDcols = -c("seed")])
    for (r in metarep) {
        set.seed(i + r)
        run_log_entry()

        model_Sub <- randomForest(x = training_set[, -c("output")], y = training_set$output, importance = TRUE, ntree = ntree, mtry = mtry, nperm = nperm)      
        model_Sub.path = paste0(outputs.path,models.folder,"model_",sample.type,"_seed_", i, "_rep_", r,"_size_",train_ins_oneshot,".rds")
        saveRDS(model_Sub, model_Sub.path)
        
        # write errors 
        obb_err = obb_error_func(model_Sub)     
        fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
            ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
        
        write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
        write_importance.rf(i,r,iter,model_Sub,sample.type)
        
    }
}

### Adaptive Training Set

In [16]:
adaptive_initial_data = upload_training_set(model.type,seed.focus,train_ins_Ad)

In [17]:
adaptive_initial_data

max_links,evidence,sc-bel-prop,prop-likelihood,n_init_believers,prior-mean,prior-sd,expertise_influence,output,seed
379.030421,33.093892,2.3776148,0.45018803,81.489234,0.25385494,0.55607425,0.44670360,42.97597,8
354.720810,53.770374,1.7306876,0.39097777,51.108860,0.30752116,0.74643066,0.49186844,45.43132,8
150.663956,52.282630,2.5397536,0.35723122,49.568888,0.46417893,0.20238519,0.59242957,45.53525,8
310.303729,37.939215,1.6481274,0.79260720,39.472428,0.41508232,0.49821941,0.54886696,46.41394,8
246.681021,69.824069,2.1237128,0.49174655,79.052799,0.66834203,0.87004269,0.41265428,43.12122,8
284.665309,2.769528,4.5423103,0.62618815,36.565307,0.37020836,0.42145825,0.70496254,46.85059,8
68.156233,17.351079,3.1303719,0.69802720,46.491671,0.35416725,0.72813793,0.35974346,45.94112,8
395.927951,72.373656,4.0555379,0.25140501,54.704373,0.74388255,0.07464179,0.76934667,45.96783,8
265.703127,79.724977,3.9051546,0.71919613,60.588566,0.21410364,0.32198382,0.32189261,47.28248,8
223.475741,60.791212,3.5232455,0.99764996,11.128286,0.93562664,0.33789561,0.36937890,53.15028,8


### Random Sampling Train & Test Metamodel

In [18]:
sample.type = "Rd"
sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

for (i in seed.focus) {
    print(paste0("seed : ", i, "  Random Sampling section start time : ", Sys.time()))
    
    for (r in metarep) { #replications
        set.seed(i + r) # set seed to control randomness
        print(paste0("seed : ", i,"   rep : ", r, "  Random Sampling section start time : ", Sys.time()))
        
        training_set_Ad = copy(adaptive_initial_data[seed == i, .SD, .SDcols = -c("seed")])       
        train_candidates_table = data.table()
            
        iter = 1
        while(iter <= iteration_budget){
            print(iter)
            run_log_entry()
            
            trainx = training_set_Ad[, .SD, .SDcols = feature_names]
            trainy = training_set_Ad$output
            
            # Train the model
            model_Sub <- randomForest(x = trainx, y = trainy, importance = TRUE,ntree = ntree, mtry = mtry, nperm = nperm)
            model_Sub.name = paste0("model_",sample.type,"_", iter, "_seed_", i, "_rep_",r)
            assign(model_Sub.name, model_Sub)
            model_Sub.path = paste0(outputs.path,models.folder, paste0(model_Sub.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                saveRDS(model_Sub, model_Sub.path)
            
            # write errors 
            obb_err = obb_error_func(model_Sub)     
            fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
                   ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
            
            write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
            write_importance.rf(i,r,iter,model_Sub,sample.type)#last one=sample_type

                   
            if (iter != iteration_budget) {
                ## sample selection from unlabeled data select candidates
                 
                unlabeled_set <- refresh_sample_pool(i + r + iter)
                train_candidates = random_sample_selection(selected_ins, unlabeled_set)
                                
                # run ABM to find outputs of train candidates
                print(paste0("ABM train_candidate run start time : ", Sys.time()))
                train_candidates = run_ABM(nofrep, selected_ins, train_candidates)
                print(paste0("ABM train_candidate run end time : ", Sys.time()))
                
                
                fwrite(data.table(train_candidates, "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_train_candidates_table_",sample.type,".csv"),append = TRUE )               
 
                # Add new data to train data
                training_set_Ad = rbind(training_set_Ad, train_candidates[, -c("idx")],use.names = TRUE)
            }
            iter = iter + 1
        }  
        
        fwrite(data.table(training_set_Ad, "seed" = i,"rep" = r),paste0(outputs.path,sample.folder,model.type,"_FinalTrainData_",sample.type,".csv") ,append = TRUE)
    
        print(paste0("seed : ", i,"   rep : ", r, "  Random Sampling section end time : ", Sys.time()))        
    }  
         
    print(paste0("seed : ", i, "  Random Sampling section end time : ", Sys.time()))
}

[1] "seed : 8  Random Sampling section start time : 2022-01-07 15:57:24"
[1] "seed : 8   rep : 1  Random Sampling section start time : 2022-01-07 15:57:24"
[1] 1
[1] "ABM train_candidate run start time : 2022-01-07 15:57:24"
[1] "ABM train_candidate run end time : 2022-01-07 16:00:40"
[1] 2
[1] "ABM train_candidate run start time : 2022-01-07 16:00:40"
[1] "ABM train_candidate run end time : 2022-01-07 16:02:47"
[1] 3
[1] "ABM train_candidate run start time : 2022-01-07 16:02:47"
[1] "ABM train_candidate run end time : 2022-01-07 16:05:51"
[1] 4
[1] "ABM train_candidate run start time : 2022-01-07 16:05:51"
[1] "ABM train_candidate run end time : 2022-01-07 16:08:20"
[1] 5
[1] "ABM train_candidate run start time : 2022-01-07 16:08:20"
[1] "ABM train_candidate run end time : 2022-01-07 16:11:02"
[1] 6
[1] "ABM train_candidate run start time : 2022-01-07 16:11:02"
[1] "ABM train_candidate run end time : 2022-01-07 16:13:51"
[1] 7
[1] "ABM train_candidate run start time : 2022-01-07 16:13

### Adaptive Train & Test Metamodel

In [19]:
adaptive_initial_data = upload_training_set(model.type,seed.focus,train_ins_Ad)

In [20]:
sample.type = paste0("Ad_",selection_metric)
sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

for(i in seed.focus){

    print(paste0("seed : ",i,"  Adaptive Sampling with ",selection_metric,"  section start time : ",Sys.time()))
    
    for (r in metarep){ #replications
        set.seed(i + r)
        print(paste0("seed : ", i,"   rep : ", r, "  Adaptive Sampling section start time : ", Sys.time()))
            
        training_set_Ad = copy(adaptive_initial_data[seed == i, .SD, .SDcols = -c("seed")])
        train_candidates_table = data.table()

        iter = 1
        while(iter <= iteration_budget){   
            print(iter)
            run_log_entry() 
        
            trainx = training_set_Ad[,.SD, .SDcols = feature_names]
            trainy = training_set_Ad$output
        
            # Train the model
            model_Sub <- randomForest( x = trainx, y =  trainy,importance = TRUE,ntree = ntree, mtry = mtry,nperm = nperm)
            model_Sub.name = paste0("model_",sample.type,"_", iter, "_seed_", i, "_rep_",r)
            model_Sub.path = paste0(outputs.path,models.folder, paste0(model_Sub.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
            saveRDS(model_Sub, model_Sub.path)
        
            # write errors 
            obb_err = obb_error_func(model_Sub)     
            fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
                   ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
        
            write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
            write_importance.rf(i,r,iter,model_Sub,sample.type)#last one=sample_type
        
            if(iter != iteration_budget){ # below efforts are unnecessary when the budget is reached.    
                ## sample selection from unlabeled data select candidates
                unlabeled_set <- refresh_sample_pool(i + r + iter)
            
                train_candidates = sample_selection(selected_ins, unlabeled_set, model_Sub,selection_metric)
            
                # run ABM to find outputs of train candidates
                print(paste0("ABM train_candidate run start time : ",Sys.time()))
                train_candidates = run_ABM(nofrep, selected_ins, train_candidates)
                print(paste0("ABM train_candidate run end time : ",Sys.time()))
            
                fwrite(data.table(train_candidates, "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_train_candidates_table_",sample.type,".csv"),append = TRUE )      

                # add labeled candidates to the train data
                training_set_Ad = rbind(training_set_Ad, train_candidates[, -c("idx")],use.names = TRUE)
            }  
        
            iter = iter + 1
        }
        fwrite(data.table(training_set_Ad, "seed" = i,"rep" = r),paste0(outputs.path,sample.folder,model.type,"_FinalTrainData_",sample.type,".csv") ,append = TRUE)
    
        print(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling section end time : ",Sys.time()))
    }
        
    print(paste0("seed : ",i,"  Adaptive Sampling section end time : ",Sys.time()))
}

[1] "seed : 8  Adaptive Sampling with coefvar  section start time : 2022-01-07 16:44:56"
[1] "seed : 8   rep : 1  Adaptive Sampling section start time : 2022-01-07 16:44:56"
[1] 1
[1] "ABM train_candidate run start time : 2022-01-07 16:44:56"
[1] "ABM train_candidate run end time : 2022-01-07 16:47:52"
[1] 2
[1] "ABM train_candidate run start time : 2022-01-07 16:47:52"
[1] "ABM train_candidate run end time : 2022-01-07 16:50:47"
[1] 3
[1] "ABM train_candidate run start time : 2022-01-07 16:50:47"
[1] "ABM train_candidate run end time : 2022-01-07 16:56:41"
[1] 4
[1] "ABM train_candidate run start time : 2022-01-07 16:56:41"
[1] "ABM train_candidate run end time : 2022-01-07 17:05:40"
[1] 5
[1] "ABM train_candidate run start time : 2022-01-07 17:05:40"
[1] "ABM train_candidate run end time : 2022-01-07 17:12:05"
[1] 6
[1] "ABM train_candidate run start time : 2022-01-07 17:12:05"
[1] "ABM train_candidate run end time : 2022-01-07 17:14:50"
[1] 7
[1] "ABM train_candidate run start time 

## Quit NL

In [21]:
NLQuit(nl.obj = nl.model)
#NLQuit(all=TRUE)